In [10]:
from shapely import Polygon
from gllm.utils.plot_utils import parse_gcode
from shapely.measurement import area, bounds
from shapely.predicates import is_closed, is_geometry, equals, is_valid_reason


In [ ]:
def evaluate_gcode(gcode_string):
    # ... (Get shape information from the user)

    # --- Polygon Input ---
    while True:
        try:
            polygon_points_str = input("Enter polygon vertices as x,y pairs separated by spaces (e.g., '0,0 10,0 10,10 0,10'): ")
            polygon_points = [(float(p.split(',')[0]), float(p.split(',')[1])) for p in polygon_points_str.split()]
            shape = Polygon(polygon_points)
            if not shape.is_valid:
                raise ValueError("Invalid polygon. Ensure it doesn't self-intersect.")
            break  # Exit loop if polygon is valid
        except ValueError as e:
            print(f"Error: {e}")

In [ ]:
gcode_square =  f"""
G17  ; Select XY plane
G28  ; Return to home position
M03  ; Spindle on clockwise
G00 X15 Y15 Z0  ; Rapid positioning to starting point
G01 Z-2 F100  ; Move down to cutting depth
G01 X35  ; Move horizontally to create the first side
G01 Y35  ; Move vertically to create the second side
G01 X15  ; Move horizontally to create the third side
G01 Y15  ; Move back to the starting point to finish the square
G00 Z5  ; Raise the tool
M05  ; Spindle stop
G28  ; Return to home position
M30  ; Program end and reset
"""

In [2]:
# Closed-from shape
gcode1 = f"""
G17  ; Select XY plane
G28  ; Return to home position
M03  ; Spindle on clockwise
G00 X10 Y10  ; Rapid move to starting point
G01 Z-2 F100  ; Move to cutting depth
G01 X30 Y10  ; Start milling custom shape
G01 X35 Y30
G01 X5 Y30
G01 X10 Y10
G01 Z0  ; Raise the tool
G00 X0 Y0  ; Return tool to home position
M05  ; Spindle stop
M30  ; Program end and reset
"""

In [5]:
# open-form shape
gcode2 = f"""
G17
G00 Z5.0
G00 X10 Y10
G01 Z-2 F100
G01 X30 Y10
G01 X35 Y30
G01 X5 Y30
G00 Z5.0
G00 X0 Y0
M30
"""

In [12]:
# Create Shape for gcode1
x_points, y_points = [], []
x_points, y_points = parse_gcode(gcode1)
vertices1 = []

for x, y in zip(x_points, y_points):
    vertices1.append((x, y))

shape1 = Polygon(vertices1)  # Create a Shapely Polygon from the vertices
print(vertices1)
print(shape1)

rapid positioning G00 X10 Y10
G00 X10 Y10 10.0 10.0
linear interpolation G01 Z-2 F100
G01 Z-2 F100 10.0 10.0
linear interpolation G01 X30 Y10
G01 X30 Y10 10.0 10.0
linear interpolation G01 X35 Y30
G01 X35 Y30 30.0 10.0
linear interpolation G01 X5 Y30
G01 X5 Y30 35.0 30.0
linear interpolation G01 X10 Y10
G01 X10 Y10 5.0 30.0
linear interpolation G01 Z0
G01 Z0 10.0 10.0
rapid positioning G00 X0 Y0
G00 X0 Y0 0.0 0.0
[(10.0, 10.0), (10.0, 10.0), (30.0, 10.0), (35.0, 30.0), (5.0, 30.0), (10.0, 10.0), (10.0, 10.0), (0.0, 0.0)]
POLYGON ((10 10, 10 10, 30 10, 35 30, 5 30, 10 10, 10 10, 0 0, 10 10))


In [15]:
# Create Shape for gcode2

x_points, y_points = parse_gcode(gcode2)
vertices2 = []

for x, y in zip(x_points, y_points):
    vertices2.append((x, y))

shape2 = Polygon(vertices2)  # Create a Shapely Polygon from the vertices
print(vertices2)
print(shape2)

rapid positioning G00 Z5.0
G00 Z5.0 0 0
rapid positioning G00 X10 Y10
G00 X10 Y10 10.0 10.0
linear interpolation G01 Z-2 F100
G01 Z-2 F100 10.0 10.0
linear interpolation G01 X30 Y10
G01 X30 Y10 10.0 10.0
linear interpolation G01 X35 Y30
G01 X35 Y30 30.0 10.0
linear interpolation G01 X5 Y30
G01 X5 Y30 35.0 30.0
rapid positioning G00 Z5.0
G00 Z5.0 5.0 30.0
rapid positioning G00 X0 Y0
G00 X0 Y0 0.0 0.0
[(0, 0), (10.0, 10.0), (10.0, 10.0), (30.0, 10.0), (35.0, 30.0), (5.0, 30.0), (5.0, 30.0), (0.0, 0.0)]
POLYGON ((0 0, 10 10, 10 10, 30 10, 35 30, 5 30, 5 30, 0 0))


In [11]:
print("shape 1: ", is_valid_reason(shape1))
print("shape 2: ", is_valid_reason(shape2))

shape 1:  Ring Self-intersection[10 10]
shape 2:  Valid Geometry


In [4]:
print(shape1)

POLYGON ((10 10, 10 10, 30 10, 35 30, 5 30, 10 10, 10 10, 0 0, 10 10))


In [8]:
print(shape2)

POLYGON ((0 0, 10 10, 10 10, 30 10, 35 30, 5 30, 5 30, 0 0))


In [9]:
print(vertices2)

[(0, 0), (10.0, 10.0), (10.0, 10.0), (30.0, 10.0), (35.0, 30.0), (5.0, 30.0), (5.0, 30.0), (0.0, 0.0)]
